In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib.gridspec import GridSpec
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from warnings import filterwarnings
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram
from factor_analyzer import FactorAnalyzer
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from itertools import combinations
from kmodes.kprototypes import KPrototypes
from collections import Counter
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.cluster import DBSCAN
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

C:\Users\Yehan Perera\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df_1 = pd.read_csv("zomato.csv")

## Capture the categories that helps to predict y more accurately 

In [3]:
def preprocessing_initial(data):
    ### Remove unwanted columns 
    col_to_remove = ['url','address','phone','reviews_list','menu_item']
    df = data.drop(columns=col_to_remove)
    
    # remove duplicates
    df.drop_duplicates(inplace=True)
    

    def fun(x):
        x_new = x.split(',')
        return x_new[0]+x_new[1]
    df['approx_cost(for two people)']=df['approx_cost(for two people)'].apply(lambda x: fun(x) if ',' in str(x) else x)
    df['approx_cost(for two people)'] = pd.to_numeric(df['approx_cost(for two people)'])
    
    
    # rate 
    df['rate'] = df['rate'].apply(lambda x : str(x).split('/')[0])
    ls = [float(i) for i in df['rate'] if (i!='NEW') & (i!='nan') & (i!='-')]
    mean_rate = round(np.mean(ls),1)
    df['rate'] = df['rate'].apply(lambda x: mean_rate if (x=='NEW') | (x=='-') | (x=='nan') else x)
    df['rate'] = pd.to_numeric(df['rate'])
    ## Remove the missing values of response 
    df.dropna(subset=['approx_cost(for two people)'], inplace=True)
    
     #########   categorical variables 
    vars_to_impute = ['location','rest_type','dish_liked','cuisines']
    for col in vars_to_impute:
        df.dropna(subset=[col],inplace=True)
        
        
    ## Split the data 
    X= df.drop(columns=['approx_cost(for two people)'])
    Y = df['approx_cost(for two people)']
   
    return   X ,Y 
    

In [4]:
X , Y = preprocessing_initial(df_1)

In [5]:
from category_encoders import TargetEncoder

In [6]:
X_train , X_test , y_train , y_test = train_test_split(X,Y,test_size=0.2,random_state=43)

In [7]:
from sklearn.preprocessing import OneHotEncoder
# Define your target encoder columns
target_encoder_cols = ['name', 'dish_liked', 'cuisines', 'location', 'rest_type', 'listed_in(city)', 'listed_in(type)', 'online_order', 'book_table']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform frequency encoding for each column
for col in target_encoder_cols:
    target_encoder = TargetEncoder()

    # Fit and transform the target encoder on the training data
    X_train[target_encoder_cols] = target_encoder.fit_transform(X_train[target_encoder_cols], y_train)

    # Transform the test data using the target encoder fitted on the training data
    X_test[target_encoder_cols] = target_encoder.transform(X_test[target_encoder_cols])


In [8]:
from sklearn.ensemble import IsolationForest

In [9]:
clf = IsolationForest(contamination=0.1)
clf.fit(X_train)
predictions = clf.predict(X_train)

* predictions==1 : cluster 2

In [10]:
sum(predictions==1)

16842

* predictions==-1 : outliers : cluster 1 

In [11]:
sum(predictions==-1)

1872

In [12]:
X_train['cluster_label'] = predictions


In [13]:
X_train = X_train[X_train['cluster_label']== 1]
indexes_1 = X_train.index
y_train = y_train.loc[indexes_1]
X_train.drop(columns=['cluster_label'],inplace=True)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

# Create Pipeline with feature selection and ElasticNet
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(ElasticNet())),
    ('elastic_net', ElasticNet())
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'feature_selection__estimator__l1_ratio': [0.1, 0.5, 0.9],
    'elastic_net__alpha': [0.1, 1.0, 10.0],
    'elastic_net__l1_ratio': [0.1, 0.5, 0.9]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
selected_features_1 = X_train.columns[best_model.named_steps['feature_selection'].get_support()]


selected_features = X_train.columns[best_model.named_steps['feature_selection'].get_support()]
# Evaluate the best model
mape_train = np.mean(np.abs((y_train - best_model.predict(X_train)) / y_train)) * 100
mape_test = np.mean(np.abs((y_test - best_model.predict(X_test)) / y_test)) * 100

print("Mean Absolute Percentage Error on Training set:", mape_train)
print("Mean Absolute Percentage Error on Testing set:", mape_test)

print(grid_search.best_params_)


Mean Absolute Percentage Error on Training set: 24.889517373175345
Mean Absolute Percentage Error on Testing set: 27.869743076525637
{'elastic_net__alpha': 0.1, 'elastic_net__l1_ratio': 0.9, 'feature_selection__estimator__alpha': 1.0, 'feature_selection__estimator__l1_ratio': 0.1}


In [15]:
selected_features

Index(['name', 'rate', 'dish_liked'], dtype='object')

In [16]:
# Calculate predictions for both training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Print MSE and RMSE for both training and testing sets
print("Mean Squared Error on Training set:", mse_train)
print("Mean Squared Error on Testing set:", mse_test)
print("Root Mean Squared Error on Training set:", rmse_train)
print("Root Mean Squared Error on Testing set:", rmse_test)

Mean Squared Error on Training set: 22922.780359754823
Mean Squared Error on Testing set: 55611.81193163657
Root Mean Squared Error on Training set: 151.402709221978
Root Mean Squared Error on Testing set: 235.82156799503426


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge  # Change this import
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

# Create Pipeline with feature selection and Ridge regression
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(Ridge())),  # Change SelectFromModel
    ('ridge', Ridge())  # Change Ridge
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'feature_selection__estimator__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Add solver options
    'ridge__alpha': [0.1, 1.0, 10.0]  # Change parameter name
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
selected_features_1 = X_train.columns[best_model.named_steps['feature_selection'].get_support()]

# Evaluate the best model
mape_train = np.mean(np.abs((y_train - best_model.predict(X_train)) / y_train)) * 100
mape_test = np.mean(np.abs((y_test - best_model.predict(X_test)) / y_test)) * 100

print("Mean Absolute Percentage Error on Training set:", mape_train)
print("Mean Absolute Percentage Error on Testing set:", mape_test)

print(grid_search.best_params_)



Mean Absolute Percentage Error on Training set: 24.903982949359428
Mean Absolute Percentage Error on Testing set: 27.883032816721325
{'feature_selection__estimator__alpha': 0.1, 'feature_selection__estimator__solver': 'sag', 'ridge__alpha': 1.0}


In [18]:
from sklearn.metrics import mean_squared_error

# Calculate predictions for both training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Print MSE and RMSE for both training and testing sets
print("Mean Squared Error on Training set:", mse_train)
print("Mean Squared Error on Testing set:", mse_test)
print("Root Mean Squared Error on Training set:", rmse_train)
print("Root Mean Squared Error on Testing set:", rmse_test)

Mean Squared Error on Training set: 22921.280288173413
Mean Squared Error on Testing set: 55613.65446565868
Root Mean Squared Error on Training set: 151.3977552283171
Root Mean Squared Error on Testing set: 235.8254745901271


In [19]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Create Pipeline with feature selection and XGBoost
pipeline_xgb = Pipeline([
    ('feature_selection', SelectFromModel(XGBRegressor())),
    ('xgb', XGBRegressor())
])

# Define parameter grid for hyperparameter tuning for XGBoost
param_grid_xgb = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'feature_selection__estimator__max_iter': [1000, 5000, 10000],
    'xgb__learning_rate': [0.1, 0.01, 0.001],
    'xgb__n_estimators': [100, 500, 1000],
    'xgb__max_depth': [3, 5, 7]
}

# Perform GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV for XGBoost
grid_search_xgb.fit(X_train, y_train)

# Get the best model for XGBoost
best_model_xgb_2 = grid_search_xgb.best_estimator_

# Get the selected features for XGBoost
selected_features_xgb = X_train.columns[best_model_xgb_2.named_steps['feature_selection'].get_support()]

# Evaluate the best model for XGBoost
mape_train_xgb = np.mean(np.abs((y_train - best_model_xgb_2.predict(X_train)) / y_train)) * 100
mape_test_xgb = np.mean(np.abs((y_test - best_model_xgb_2.predict(X_test)) / y_test)) * 100

print("Selected Features (XGBoost):", selected_features_xgb)
print("Mean Absolute Percentage Error on Training set (XGBoost):", mape_train_xgb)
print("Mean Absolute Percentage Error on Testing set (XGBoost):", mape_test_xgb)

Selected Features (XGBoost): Index(['dish_liked'], dtype='object')
Mean Absolute Percentage Error on Training set (XGBoost): 8.194567865722085
Mean Absolute Percentage Error on Testing set (XGBoost): 14.519004442554845


In [20]:
y_train_pred_xgb = best_model_xgb_2.predict(X_train)
y_test_pred_xgb = best_model_xgb_2.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train_xgb = mean_squared_error(y_train, y_train_pred_xgb)
mse_test_xgb = mean_squared_error(y_test, y_test_pred_xgb)

# Calculate Root Mean Squared Error (RMSE)
rmse_train_xgb = np.sqrt(mse_train_xgb)
rmse_test_xgb = np.sqrt(mse_test_xgb)

# Print MSE and RMSE for both training and testing sets
print("Mean Squared Error on Training set (XGBoost):", mse_train_xgb)
print("Mean Squared Error on Testing set (XGBoost):", mse_test_xgb)
print("Root Mean Squared Error on Training set (XGBoost):", rmse_train_xgb)
print("Root Mean Squared Error on Testing set (XGBoost):", rmse_test_xgb)

Mean Squared Error on Training set (XGBoost): 8703.028534778436
Mean Squared Error on Testing set (XGBoost): 93422.37152354191
Root Mean Squared Error on Training set (XGBoost): 93.29002376877409
Root Mean Squared Error on Testing set (XGBoost): 305.650734537874


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

# Create Pipeline with feature selection and Lasso regression
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(Lasso())),
    ('lasso', Lasso())
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'lasso__alpha': [0.1, 1.0, 10.0]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
selected_features = X_train.columns[best_model.named_steps['feature_selection'].get_support()]

# Evaluate the best model
mape_train = np.mean(np.abs((y_train - best_model.predict(X_train)) / y_train)) * 100
mape_test = np.mean(np.abs((y_test - best_model.predict(X_test)) / y_test)) * 100

print("Mean Absolute Percentage Error on Training set:", mape_train)
print("Mean Absolute Percentage Error on Testing set:", mape_test)

print(grid_search.best_params_)


Mean Absolute Percentage Error on Training set: 20.963511293986027
Mean Absolute Percentage Error on Testing set: 22.843398049195308
{'feature_selection__estimator__alpha': 0.1, 'lasso__alpha': 0.1}


In [22]:
# Calculate predictions for both training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Print MSE and RMSE for both training and testing sets
print("Mean Squared Error on Training set:", mse_train)
print("Mean Squared Error on Testing set:", mse_test)
print("Root Mean Squared Error on Training set:", rmse_train)
print("Root Mean Squared Error on Testing set:", rmse_test)

Mean Squared Error on Training set: 17861.527375804737
Mean Squared Error on Testing set: 38599.439189848
Root Mean Squared Error on Training set: 133.64702531595955
Root Mean Squared Error on Testing set: 196.4673998144425


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

# Create Pipeline with feature selection and Random Forest regression
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(RandomForestRegressor())),
    ('rf', RandomForestRegressor())
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'rf__n_estimators': [50, 100, 200, 300],
    'rf__max_depth': [None, 5, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['auto', 'sqrt', 'log2']}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
selected_features = X_train.columns[best_model.named_steps['feature_selection'].get_support()]

# Evaluate the best model
mape_train = np.mean(np.abs((y_train - best_model.predict(X_train)) / y_train)) * 100
mape_test = np.mean(np.abs((y_test - best_model.predict(X_test)) / y_test)) * 100

print("Mean Absolute Percentage Error on Training set:", mape_train)
print("Mean Absolute Percentage Error on Testing set:", mape_test)

print(grid_search.best_params_)


Mean Absolute Percentage Error on Training set: 0.6518409307288845
Mean Absolute Percentage Error on Testing set: 7.726569795578484
{'rf__max_depth': None, 'rf__max_features': 'log2', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}


In [24]:
# Calculate predictions for both training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Calculate Mean Squared Error (MSE)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate Root Mean Squared Error (RMSE)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Print MSE and RMSE for both training and testing sets
print("Mean Squared Error on Training set:", mse_train)
print("Mean Squared Error on Testing set:", mse_test)
print("Root Mean Squared Error on Training set:", rmse_train)
print("Root Mean Squared Error on Testing set:", rmse_test)

Mean Squared Error on Training set: 264.44352124256216
Mean Squared Error on Testing set: 40074.72298407441
Root Mean Squared Error on Training set: 16.261719504485438
Root Mean Squared Error on Testing set: 200.18672029901086
